# Eating out in Lyon - Data gathering

## Summary

This is the first part of a data analysis project focused on finding a delicious pizza in my hometown of Lyon. Follow me as I:
- extract restaurant ratings using the yelp fusion API and the request package,
- transform the json response to a tidy dataframe,
- ensure the data integrity of variables of interest.

In [1]:
# loading all necessary packages at once
import requests
import json
import pandas as pd
import numpy as np

## Gathering restaurant data from the yelp API

The yelp API allows to get 50 results at a time up to the first 1000 results of a given query.
To overcome this limitation, we can first get all the restaurant subcategories (_e.g._, italian) and then loop over all categories to get up to 1000 restaurants by categories.

### Fetching all restaurant subcategories:

I first get the categories using the 'categories' endpoint:

In [2]:
# loading my yelp API key
key_file = open('temp_api_key.txt')
api_key = key_file.readline().rstrip('\n')
key_file.close()
# replace the above lines with your own key:
# api_key = <your api key>

# specifying the headers, endpoint and parameters:
headers = {'Authorization': 'bearer %s' % api_key}

endpoint = "https://api.yelp.com/v3/categories"

parameters = {'location': 'Lyon, FR'}

# let the request package build the url and get a response:
response = requests.get(url = endpoint, params = parameters, headers = headers)

In [3]:
# transform the response into a data frame
categories_df = pd.json_normalize(response.json()['categories'])
categories_df.head()

,alias,title,parent_aliases,country_whitelist,country_blacklist
0,3dprinting,3D Printing,[localservices],[],[]
1,abruzzese,Abruzzese,[italian],[IT],[]
2,absinthebars,Absinthe Bars,[bars],[CZ],[]
3,acaibowls,Acai Bowls,[food],[],"[AR, CL, IT, MX, PL, TR]"
4,accessories,Accessories,[fashion],[],[]


As you can see above, I now have a dataframe with all the yelp categories. Most of them have nothing to do with food, such as the first one ('3dprinting'). To keep only categories of interest, I filter the data frame to keep only the rows whose parent categories ('parent_aliases' column) contains 'restaurants':

In [4]:
# creating a column that states whether the row is a sub-category of 'restaurants':
categories_df['is_restaurant'] = ['restaurants' in parent for parent in categories_df['parent_aliases']]

In [5]:
# filtering by the 'is_restaurant' column:
restaurants_df = categories_df[categories_df.is_restaurant]
restaurants_df.head()

,alias,title,parent_aliases,country_whitelist,country_blacklist,is_restaurant
18,afghani,Afghan,[restaurants],[],"[MX, TR]",True
19,african,African,[restaurants],[],[TR],True
39,andalusian,Andalusian,[restaurants],"[ES, IT]",[],True
53,arabian,Arabic,[restaurants],[],[DK],True
59,argentine,Argentine,[restaurants],[],[FI],True


### Finding all restaurants in Lyon

We can now fetch restaurants information by looping over all categories, using the 'businesses/search' endpoint of the yelp API. I first declare some parameters by the query:

In [6]:
key_file = open('temp_api_key.txt')
api_key = key_file.readline().rstrip('\n')
key_file.close()

headers = {'Authorization': 'bearer %s' % api_key}

endpoint = "https://api.yelp.com/v3/businesses/search"

parameters = {'location': 'Lyon, FR', # where to look
             'offset' : 0, # starting from the first result
             'limit': 50, # taking 50 results (the maximum available at a time)
             'term': 'restaurants'}

I then declare a list in which to store the response and loop over categories:

In [7]:
restaurants_ratings = []

# looping over categories
for category in restaurants_df.alias:
    # specifying the category in the parameters
    parameters['categories'] = category
    # looping a second time to fetch 50 results at a time
    for offset in range(0, 1000, 50):
        parameters['offset'] = offset
        response = requests.get(url = endpoint, params = parameters, headers = headers)
        
        # we break the loop if there are no restaurants left
        if not response.json().get('businesses', False):
            break
            
        # we extend the restaurants list with the new response
        restaurants_ratings.extend(response.json()['businesses'])

In [8]:
len(restaurants_ratings)

4999

In [9]:
restaurants_ratings[0]

{'id': 'D3NHTerar80aeR6mlyE2mw',
 'alias': 'azur-afghan-lyon',
 'name': 'Azur Afghan',
 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/8i5nsqv5tbxxg8HdndPY4Q/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/azur-afghan-lyon?adjust_creative=qbeDf2GYKB1Prc0VgyQp0A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=qbeDf2GYKB1Prc0VgyQp0A',
 'review_count': 23,
 'categories': [{'alias': 'afghani', 'title': 'Afghan'}],
 'rating': 4.0,
 'coordinates': {'latitude': 45.77502, 'longitude': 4.82875},
 'transactions': [],
 'price': '€€',
 'location': {'address1': '6 Rue Villeneuve',
  'address2': None,
  'address3': None,
  'city': 'Lyon',
  'zip_code': '69004',
  'country': 'FR',
  'state': '69',
  'display_address': ['6 Rue Villeneuve', '69004 Lyon', 'France']},
 'phone': '+33478396619',
 'display_phone': '+33 4 78 39 66 19',
 'distance': 1845.795955776875}

## Data cleaning

We gathered the data from 4995 restaurants in a json format. We can first transform it to a data frame using the 'json_normalize' function which deals fairly well with nested jsons:

In [10]:
data = pd.json_normalize(restaurants_ratings)

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4999 entries, 0 to 4998
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        4999 non-null   object 
 1   alias                     4999 non-null   object 
 2   name                      4999 non-null   object 
 3   image_url                 4999 non-null   object 
 4   is_closed                 4999 non-null   bool   
 5   url                       4999 non-null   object 
 6   review_count              4999 non-null   int64  
 7   categories                4999 non-null   object 
 8   rating                    4999 non-null   float64
 9   transactions              4999 non-null   object 
 10  price                     2681 non-null   object 
 11  phone                     4999 non-null   object 
 12  display_phone             4999 non-null   object 
 13  distance                  4999 non-null   float64
 14  coordina

We can already drop columns that don't interest us:

In [12]:
# droping useless columns:
data.drop(['image_url',
          'is_closed', #
          'url',
          'transactions',
          'phone',
          'display_phone',
          'distance',
          'location.state' # redundant with location.zip_code
          ],
         axis = 1, inplace = True)

Some duplicates might have gotten into the data, for example if a restaurant was found both under the 'pizza' and 'italian' aliases. We get rid of them by dropping rows that share the same restaurant id:

In [13]:
data.drop_duplicates(subset = 'id', inplace = True)

Let's check that all the restaurants are in Lyon, France:

In [14]:
data['location.country'].value_counts()

FR    3592
IT       1
Name: location.country, dtype: int64

In [15]:
data['location.city'].value_counts()

Lyon                2628
Villeurbanne         287
Bron                  53
Oullins               44
Vénissieux            44
                    ... 
Lyon  Eme              1
Rillieux-La-Pape       1
Paris                  1
LYON 03                1
Oullins Cedex          1
Name: location.city, Length: 131, dtype: int64

It seems that one italian restaurants and quite a lot of restaurants from cities near Lyon have gotten into our data, so let's filter by city:

In [16]:
# lowering the city field to make sure we don't exlude any restaurants due to case issues
data['location.city'] = data['location.city'].str.lower()
data = data[data['location.city'].str.find('lyon') >= 0]

In [17]:
# checking that all restaurants are now in lyon
data['location.city'].value_counts()

lyon                      2630
lyon 03                      8
lyon 06                      8
lyon 07                      7
lyon 2eme                    7
lyon 6eme                    6
sainte-foy-lès-lyon          6
lyon 9eme                    6
lyon 5eme                    3
lyon-2e-arrondissement       3
lyon 02                      3
lyon 1er                     3
sainte foy les lyon          2
lyon 01                      2
lyon 05                      2
lyon 3eme                    2
lyon-5e-arrondissement       2
lyon-7e-arrondissement       2
lyon 08                      1
lyon cedex 3                 1
lyon  eme                    1
lyon-3e-arrondissement       1
lyon 7eme                    1
lyon 8eme                    1
lyon 3 eme                   1
lyon 04                      1
lyon 09                      1
sainte foy lès lyon          1
Name: location.city, dtype: int64

In [18]:
# excluding restaurants from 'sainte-foy-les-lyon' which is not in lyon
data = data[data['location.city'].str.find('sainte') == -1]

In [19]:
# checking that all restaurants are now in lyon
data['location.city'].value_counts()

lyon                      2630
lyon 06                      8
lyon 03                      8
lyon 07                      7
lyon 2eme                    7
lyon 6eme                    6
lyon 9eme                    6
lyon 5eme                    3
lyon 1er                     3
lyon 02                      3
lyon-2e-arrondissement       3
lyon-7e-arrondissement       2
lyon 3eme                    2
lyon-5e-arrondissement       2
lyon 05                      2
lyon 01                      2
lyon cedex 3                 1
lyon 08                      1
lyon  eme                    1
lyon-3e-arrondissement       1
lyon 7eme                    1
lyon 8eme                    1
lyon 3 eme                   1
lyon 04                      1
lyon 09                      1
Name: location.city, dtype: int64

In [20]:
# We can finally drop the city column, since it is redundant with zip codes
data.drop('location.city', axis = 1, inplace = True)

We can have a final check of our filter using zip-codes: Lyon is from 69001 to 69009, so there should be no other zipcodes:

In [21]:
data['location.zip_code'].value_counts()

69003    500
69002    477
69007    400
69006    387
69001    336
69005    215
69009    149
69008    109
69004    108
           7
69100      3
69000      3
26150      2
69300      2
69200      1
69363      1
69326      1
69800      1
69500      1
Name: location.zip_code, dtype: int64

In [22]:
# dropping restaurants with zip-codes outside of Lyon
data = data[(data['location.zip_code'].isin(['69001', '69002', '69003', '69004',
                                          '69005', '69006', '69007', '69008',
                                          '69009']))]

In [23]:
len(data)

2681

We can then inspect missing values:

In [24]:
data.isna().sum()

id                             0
alias                          0
name                           0
review_count                   0
categories                     0
rating                         0
price                       1166
coordinates.latitude           0
coordinates.longitude          0
location.address1              4
location.address2           1010
location.address3           1294
location.zip_code              0
location.country               0
location.display_address       0
dtype: int64

Missing secondary and third addresses are not an issues since most places only have a primary adress. Missing prices might be concerning if we want to later analyse ratings vs price. Lastly, we can drop the rows with a missing value in coordinates and primary address, since the only concern a few restaurants.

In [25]:
data.dropna(subset = ['coordinates.latitude', 'coordinates.longitude', 'location.address1'], inplace = True)

In [26]:
data[data.columns[:8]].head()

,id,alias,name,review_count,categories,rating,price,coordinates.latitude
0,D3NHTerar80aeR6mlyE2mw,azur-afghan-lyon,Azur Afghan,23,"[{'alias': 'afghani', 'title': 'Afghan'}]",4.0,€€,45.775020
1,ee4wtKIBI_yTz0fJD054pg,tendance-afghane-lyon,Tendance Afghane,1,"[{'alias': 'afghani', 'title': 'Afghan'}]",3.0,NaN,45.759540
2,zmk41IUwIkvO_eM0UGD7Sg,sufy-lyon,Sufy,2,"[{'alias': 'indpak', 'title': 'Indian'}, {'ali...",3.5,NaN,45.752212
3,Vo0U5EcXbh7qlpdaQwZchA,le-conakry-lyon,Le Conakry,9,"[{'alias': 'african', 'title': 'African'}]",4.0,€€,45.750642
4,-mFHJBuCxZJ_wJrO-o2Ypw,afc-africa-food-concept-lyon,AFC Africa Food Concept,8,"[{'alias': 'african', 'title': 'African'}, {'a...",3.5,€€,45.754336


In [27]:
data[data.columns[8:]].head()

,coordinates.longitude,location.address1,location.address2,location.address3,location.zip_code,location.country,location.display_address
0,4.828750,6 Rue Villeneuve,None,None,69004,FR,"[6 Rue Villeneuve, 69004 Lyon, France]"
1,4.825560,25 Rue Tramassac,,,69005,FR,"[25 Rue Tramassac, 69005 Lyon, France]"
2,4.864384,34 rue Jeanne Hachette,,None,69003,FR,"[34 rue Jeanne Hachette, 69003 Lyon, France]"
3,4.849127,112 Grande rue de la Guillotière,,,69007,FR,"[112 Grande rue de la Guillotière, 69007 Lyon,..."
4,4.843469,14 Grande rue de la Guillotière,,,69007,FR,"[14 Grande rue de la Guillotière, 69007 Lyon, ..."


In [28]:
data['categories'][2]

[{'alias': 'indpak', 'title': 'Indian'},
 {'alias': 'afghani', 'title': 'Afghan'},
 {'alias': 'buffets', 'title': 'Buffets'}]

In [29]:
def get_aliases(category_list):
    return([element['alias'] for element in category_list if element['alias'] in list(restaurants_df['alias'])])

In [30]:
data['cat_aliases'] = data['categories'].apply(get_aliases)

In [31]:
categories_table = pd.concat([data['id'], data['cat_aliases'].apply(lambda x: pd.Series(1, x)).fillna(0)], axis = 1)

In [32]:
categories_table

,id,afghani,indpak,buffets,african,halal,french,burgers,hotdogs,seafood,...,hungarian,indonesian,irish,trattorie,peruvian,latin,sud_ouest,syrian,oriental,persian
0,D3NHTerar80aeR6mlyE2mw,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ee4wtKIBI_yTz0fJD054pg,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,zmk41IUwIkvO_eM0UGD7Sg,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Vo0U5EcXbh7qlpdaQwZchA,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-mFHJBuCxZJ_wJrO-o2Ypw,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4975,lD9ECRUPZeS1l6HBdfD4BQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4977,mh5klor8OXnYYPIPz9s6TQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4978,SVa5-glOYFajWkIuETg67w,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4980,MxDyahUjN0EszfjrMJWRyw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
data.drop(['categories', 'cat_aliases'], axis = 1, inplace = True)

In [34]:
data.to_csv('data/restaurants.csv', index = False)

In [35]:
categories_table.to_csv('data/categories.csv', index = False)